In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.data.features import feature_engineering
from src.data.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.evaluator import evaluation

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

In [5]:
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

In [6]:
import pandas as pd
from src.models.candidate import candidate_generation

df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train = pd.concat([df_train.iloc[:,:3], df_train_neg], ignore_index=True)

del df_train_neg

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_valid)
    ]

In [7]:
# perform bayesian search
searcher = BayesianSearch(config['optimization']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-04-22 19:22:57,419] A new study created in memory with name: no-name-54edbb6a-b853-47ba-8ece-1e03da917c37
[I 2025-04-22 19:22:59,542] Trial 0 finished with value: 0.9474291287193659 and parameters: {'learning_rate': 0.23452112109053377, 'gamma': 4.870609588765207, 'max_depth': 4, 'subsample': 0.8835580872821923, 'n_estimators': 158}. Best is trial 0 with value: 0.9474291287193659.
[I 2025-04-22 19:23:02,227] Trial 1 finished with value: 0.9468900543790266 and parameters: {'learning_rate': 0.078710564087631, 'gamma': 0.261141610098527, 'max_depth': 6, 'subsample': 0.5835749005927385, 'n_estimators': 171}. Best is trial 0 with value: 0.9474291287193659.
[I 2025-04-22 19:23:04,754] Trial 2 finished with value: 0.9467646490442457 and parameters: {'learning_rate': 0.1856787036485623, 'gamma': 1.9013473969407624, 'max_depth': 10, 'subsample': 0.9980697997372148, 'n_estimators': 181}. Best is trial 0 with value: 0.9474291287193659.
[I 2025-04-22 19:23:06,823] Trial 3 finished with val

In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'learning_rate': 0.2299459340918077, 'gamma': 2.534174904947987, 'max_depth': 8, 'subsample': 0.5477455410545349, 'n_estimators': 206}


In [9]:
import pandas as pd

# use indicator to know the origin
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
# keep only rows that are present in df1 but not in df2
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative sampling
df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train = pd.concat([df_train, df_train_neg], ignore_index=True)

del df_train_neg

df_train, df_test = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_test)
    ]

In [10]:
from sklearn.metrics import ndcg_score
from xgboost import XGBRanker
import numpy as np

clf = XGBRanker(**study.best_trial.params)
clf.fit(
    df_train['X'], df_train['y'].astype(int)
    , group=df_train['group']
    , verbose=False
    )

preds = clf.predict(df_test['X'])

print(evaluation(df_test['y'], preds, df_test['group']))

0.9530592924293719


In [11]:
from src.models.baseline import baseline_model

result = baseline_model(dataframes=dfs, n=10)
group = result.groupby('user_id').size().to_list()

print(evaluation(result['rating'], result['est_rating'], group))
del group

0.9168812846621082


In [12]:
user_item_features = feature_engineering(dataframes={
    'user': dfs['user'], 'item': dfs['item'], 'data': dfs['data']
    })

df_baseline = build_rank_input(ratings=result.drop(columns=['est_rating']), features=user_item_features)

preds = clf.predict(df_baseline['X'])
print(evaluation(df_baseline['y'], preds, df_baseline['group']))

0.9492278320090779
